In [ ]:
from lxml import etree as ET
import pandas as pd
import re
import os
import sys

### Schritt 1: LOD-Datei einlesen und Register mit Namen erstellen

lod_file = 'hainhofer-lod.xml'
if not os.path.exists(lod_file):
    print(f"Fehler: LOD-Datei '{lod_file}' nicht gefunden.")
    sys.exit(1)

try:
    tree = ET.parse(lod_file)
    root = tree.getroot()
except ET.ParseError as e:
    print(f"Fehler beim Parsen der XML-Datei: {e}")
    sys.exit(1)

namespaces = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "schema": "http://schema.org/"
}

base_url = "https://hainhofer.hab.de/register/objekte/"

wrk_to_obj = {}

for desc in root.findall(".//rdf:Description", namespaces):
    label_elem = desc.find("rdfs:label", namespaces)
    page_elem = desc.find("schema:mainEntityOfPage", namespaces)

    if label_elem is not None and page_elem is not None:
        url = page_elem.text.strip()
        if url.startswith(base_url):
            wrk_id = url[len(base_url):].strip()
            obj_label = label_elem.text.strip()
            wrk_to_obj[wrk_id] = obj_label

print(f"{len(wrk_to_obj)} Objekte aus LOD geladen.")

### Schritt 2: TEI-XML-Bericht verarbeiten

user_input = input("Reiseziel und Jahr des Berichts eingeben (z.B. München 1603): ").strip()
input_file = user_input.title() + ".xml"

if not os.path.exists(input_file):
    print(f"Fehler: Datei '{input_file}' nicht gefunden.")
    sys.exit(1)

try:
    tree = ET.parse(input_file)
    root = tree.getroot()
except ET.ParseError as e:
    print(f"Fehler beim Parsen der TEI-XML-Datei: {e}")
    sys.exit(1)

tei_ns = {"tei": "http://www.tei-c.org/ns/1.0"}

def extract_text_recursive(elem):
    return re.sub(r'\s+', ' ', ''.join(elem.itertext())).strip()

objects_list = []

for elem in root.findall('.//tei:rs[@type="work"][@role="present"]', tei_ns):
    parent = elem.getparent()
    skip = False
    while parent is not None:
        if parent.tag == f"{{{tei_ns['tei']}}}note" and parent.attrib.get('resp') == '#editor':
            skip = True
            break
        if (parent.tag == f"{{{tei_ns['tei']}}}p" or parent.tag == f"{{{tei_ns['tei']}}}div") and parent.attrib.get('hand') == '#fremde_hand':
            skip = True
            break
        parent = parent.getparent()
    if skip:
        continue

    ref = elem.get('ref')
    if ref is None or not ref.startswith('wrk:'):
        continue

    wrk_id = ref[4:].strip()
    obj_text = extract_text_recursive(elem)
    objects_list.append((obj_text, wrk_id))

print(f"{len(objects_list)} Objekterwähnungen im Bericht gefunden.")

### Schritt 3: Erwähnungen normalisieren

normalized = []

for obj_text, wrk_id in objects_list:
    obj_name = wrk_to_obj.get(wrk_id)
    if obj_name:
        normalized.append({
            "Erwähnung": obj_text,
            "Objekt": obj_name
        })

df = pd.DataFrame(normalized)

print(df)

### Schritt 4: Ergebnisse speichern (Excel und CSV)

base_name = os.path.splitext(input_file)[0]
excel_output_path = base_name + "_objekte_normalisiert.xlsx"
csv_output_path = base_name + "_objekte_normalisiert.csv"

try:
    df.to_excel(excel_output_path, index=False)
    df.to_csv(csv_output_path, sep=";", index=False)
except Exception as e:
    print(f"Fehler beim Speichern der Dateien: {e}")
    sys.exit(1)

print(f"\nErgebnis erfolgreich gespeichert unter:\n- {excel_output_path}\n- {csv_output_path}")


123 Objekte aus LOD geladen.
165 Objekterwähnungen im Bericht gefunden.
                                           Erwähnung  \
0                                          Rathshauß   
1                                              gebew   
2                                         thum kürch   
3                                   selbiger kürchen   
4                               Sankt Peters Kirchen   
5                                   Jesuiter Kirchen   
6                                         groß altar   
7                                           Crucifix   
8                                   Sancta Magdalena   
9   messinen Crucifix, seinauch klaine messine Engel   
10                                         collegium   
11                                         collegium   
12                                               baw   
13                                         collegium   
14                                             gebew   
15                              